In [1]:
!pip install sparqlwrapper

Defaulting to user installation because normal site-packages is not writeable


DEPRECATION: Loading egg at c:\programdata\anaconda3\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [1]:
from SPARQLWrapper import SPARQLWrapper2, JSON
from string import punctuation
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import re
import datetime

nltk.download("punkt")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\fredd\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
entities = ["artifact", "event", "category", "convervation", "department", "exhibit"
,"gallery", "gift", "language", "loan", "loccation", "material", 
"membership", "nationality", "origin", "conservator", "curator", "guide",
"organiser", "security", "artist", "author", "borrower", "donor", "member", 
"sponsor", "publication", "sculpture", "specialty", "tour"]
ATTRIBS = ["name", "start", "end", "amount", "biography", "email", '']

In [3]:
userInput = input('Enter your query: ')
userInput

Enter your query: list all emails of donors


'list all emails of donors'

In [4]:
def normalize_function(userInput):
    userInput = re.sub(" +", " ", userInput)
    userInput = re.sub(f"[{re.escape(punctuation)}]", "", userInput)
    tokens = word_tokenize(userInput)
    return userInput, tokens

In [5]:
def stem_input(userInput, tokens):
    ps = PorterStemmer()
    words = [ps.stem(token) for token in tokens]
    userInput = " ".join(words)
    return userInput

In [6]:
userInput, tokens = normalize_function(userInput)
userInput = stem_input(userInput, tokens)
userInput

'list all email of donor'

In [7]:

ENTITIE = ""
if 'curat' in userInput.split():
    userInput = userInput.replace('curat', 'curator')
    
for word in userInput.split():
    if word.lower() in entities:
        ENTITIE = word
        break
        
if ENTITIE == '':
    ENTITIE = 'name'
        
ENTITIE

'donor'

In [8]:
ATTRIB = ''
for word in userInput.split():
    if word in ATTRIBS:
        ATTRIB = word
        break
        
if ATTRIB == '':
    ATTRIB = 'Name'

        
print(ATTRIB)

email


In [9]:
def Get_Filter(userInput, x):
    data = 'data goes here'
    if x == 'After':
        operand = '>'
    elif x == 'Befor':
        operand = '<'
    
    else:
        operand = '=='
        
    index = 0
    for word in userInput.title().split():
        index += 1
        if word == x:
            data = userInput.split()[index:]
            data = " ".join(data)
            break;
    return operand, data
        

In [10]:
contains_filter = False
operand = ''
data = ''

for x in ["From", "On", "After", "Befor", "In"]:
    if x in userInput.title().split():
        contains_filter = True
        operand, data =  Get_Filter(userInput, x)
        break;

In [11]:
if((contains_filter == False) and (len(data) == 0)):
    sparqlQuery = f"""
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX mus: <http://www.semanticweb.org/freddie/ontologies/2024/10/untitled-ontology-13#>

    SELECT ?{ATTRIB.lower()} WHERE {{
        ?a a mus:{ENTITIE.title()}.
        ?a mus:{ATTRIB.lower()} ?{ATTRIB.lower()}.
    }}
    """
elif operand == '==':
    data = data.title()
    sparqlQuery = f"""
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX mus: <http://www.semanticweb.org/freddie/ontologies/2024/10/untitled-ontology-13#>

    SELECT ?{ATTRIB.title()} WHERE {{
    ?a a mus:{ENTITIE.title()}.
    ?b a mus:Category.
    ?a mus:{ATTRIB.title()} ?{ATTRIB.title()}.
    ?a mus:Category_ID ?catID.
    ?b mus:Category_ID ?catID.  
    ?b mus:Name ?catName.      
    FILTER(?catName = \'{data}\')
}}
    """
else:
    data = f"{data[:2]}-{data[2:4]}-{data[4:]}"
    data = datetime.datetime.strptime(data, '%d-%m-%Y').date()
    print(data)
    sparqlQuery = f"""
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX mus: <http://www.semanticweb.org/freddie/ontologies/2024/10/untitled-ontology-13#>

    SELECT ?{ATTRIB.title()} WHERE {{
    ?a a mus:{ENTITIE.title()}.
    ?a mus:{ATTRIB.title()} ?{ATTRIB.title()}.
    ?a mus:Start_Date ?start_date.     
    FILTER(?start_date {operand} \"{data}T00:00:00\"^^xsd:dateTime)
}}
    """
    

print(sparqlQuery)

sparql = SPARQLWrapper2("http://localhost:3030/museum-collection/sparql")
sparql.setQuery(sparqlQuery)
results = sparql.query().bindings
    


    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX mus: <http://www.semanticweb.org/freddie/ontologies/2024/10/untitled-ontology-13#>

    SELECT ?email WHERE {
        ?a a mus:Donor.
        ?a mus:email ?email.
    }
    


In [12]:
for result in results:
    print(result[ATTRIB.lower()].value)

LM@gmail.com
NT@gmail.com
MR@gmail.com
LC@gmail.com
SN@gmail.com
MR@gmail.com
ES@gmail.com
AS@gmail.com
LM@gmail.com
ZB@gmail.com
JT@gmail.com
LG@gmail.com
AP@gmail.com
OW@gmail.com
IC@gmail.com
JB@gmail.com
HG@gmail.com
EC@gmail.com
CF@gmail.com
